In [1]:
%pip install awswrangler --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import awswrangler as wr
import pandas as pd

In [3]:
lowg_query = 's3://universityose-bucket/yvlcmb/LowG/2023/10/07/7e98c65e-1bbd-4f31-9785-1150242ce584.csv'

In [4]:
df = wr.s3.read_csv(
    boto3_session=boto3.Session(), 
    path=lowg_query, 
    path_suffix='.csv',
    skip_blank_lines=True)

In [5]:
df['times_gpsdatetime'] = pd.to_datetime(df.times_gpsdatetime)
df.columns =  ['gps_time', 'pitch', 'roll', 'speed', 'descent']
df.iloc[[-1, 0]].gps_time.diff()

157807                      NaT
0        0 days 00:00:40.605000
Name: gps_time, dtype: timedelta64[ns]

In [7]:
# just calling the .diff() method on each column besides `gps_time`
diff_cols = [{'_'.join((col, 'diff')): df[col].diff() for col in df.columns.to_list()[1:]}]

In [8]:
# adding the columns with the row-by-row differences to the dataframe
for data in diff_cols:
    for key, val in data.items(): 
         df[key] = pd.Series(val)

In [9]:
df.tail()

,gps_time,pitch,roll,speed,descent,pitch_diff,roll_diff,speed_diff,descent_diff
157803,2023-06-15 20:01:39.518,-4.090,-6.282,5.958612,-4.282,0.005,-0.014,0.003555,0.258
157804,2023-06-15 20:01:39.535,-4.086,-6.295,5.962166,-4.011,0.004,-0.013,0.003555,0.271
157805,2023-06-15 20:01:39.552,-4.082,-6.308,5.965721,-3.710,0.004,-0.013,0.003555,0.301
157806,2023-06-15 20:01:39.579,-4.078,-6.333,5.972239,-3.073,0.004,-0.025,0.006517,0.637
157807,2023-06-15 20:01:39.597,-4.076,-6.342,5.974016,-2.822,0.002,-0.009,0.001777,0.251


In [10]:
rows_per_minute = len(df)/40.605

In [11]:
print(round(rows_per_minute, 3))

3886.418


In [12]:
rows_per_second = rows_per_minute / 60

In [13]:
print(round(rows_per_second,3))

64.774
